In [ ]:
import pandas as pd
import numpy as np
import pyvista as pv
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Introduction

In this notebook, you should apply the techniques you learned for regression in a more realistic problem setting. We have a collection of bridges modeled as 2D beams that all feature one defect. Our goal is to train a model to learn the location of this defect as a function of displacement measurements. Since sensors are expensive, we can only place them in five locations on the bridges. The assignment includes the following tasks:

- Select five locations for the sensors based on a visual inspection of the displacement field 
- Train a neural network to learn a mapping from the displacement measurements to the defect location, comment on the choice of hyperparameters (number of hidden layers, nodes per layer, ...)
- Visualise your results and evaluate the accuracy of your network

![overview beam structure](img/beam_structure.png)


Let's take a look at the dataset first. It is a CSV file, and a convenient way to read and manipulate this file type is via the `Dataframe` of the `pandas` library. Printing a few lines of the dataset before performing the analysis is good practice. We load the dataset into a `Dataframe` from the `pandas` library and print a few rows from the top and bottom. The dataset consists of a collection of displacement fields of the bridges. We have a total of 1000 bridges, as can be seen from the tail of the data frame `df.tail()`, and 712 locations in which the displacements have been measured, as can be seen from the tail of a sample `bar_0.tail()`, which is just a single sample we took from the dataset. Note that some of the columns, such as E_pure, nu, and location, are uniform for a specific sample owing to the dataset's structure.

In [ ]:
df = pd.read_csv('regression-data.csv')
bar_0 = df[df['sample'] == 0]
df.head()

In [ ]:
df.tail()

In [ ]:
bar_0.tail()

## Data visualization and feature extraction

Your first task is to select measurement locations. The following cell plots the displacement in the x- or y-direction or the magnitude over the beam's domain. You can select the three components via the buttons on the topside. In addition, you can see all the available measurement locations. You can hover over the plot, which will display the data frame's corresponding node ID.

In [ ]:
# extract data corresponding to one single bridge, and interpolate the displacements on a grid for plotting
bar_0 = df[df['sample'] == 0]
grid_x, grid_y = np.mgrid[0.02:9.98:250j, 0.02:1.98:50j]
grid_z = griddata(bar_0[['x','y']].to_numpy(), np.sqrt(bar_0['dx']**2 + bar_0['dy']**2), (grid_x, grid_y))

# plot displacement-field and nodes
fig = go.Figure()
fig.add_trace(go.Heatmap(z=grid_z.transpose(),
                         x=grid_x[:,0],
                         y=grid_y[0],
                         hoverinfo='skip',
                         name='heatmap'))

# plot nodes
fig.add_trace(go.Scatter(x=bar_0['x'],
                         y=bar_0['y'],
                         mode='markers',
                         marker_color='black',
                         name='',
                         hovertemplate='<b>Node</b>: %{text}',
                         text=bar_0['node']))

# add buttons to display different displacement fields
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=['z', [griddata(bar_0[['x','y']].to_numpy(), np.sqrt(bar_0['dx']**2 + bar_0['dy']**2), (grid_x, grid_y)).transpose()]],
                    label='magnitude', method='restyle'),
                dict(
                    args=['z', [griddata(bar_0[['x','y']].to_numpy(), bar_0['dx'], (grid_x, grid_y)).transpose()]],
                    label='x',
                    method='restyle'),
                dict(
                    args=['z', [griddata(bar_0[['x','y']].to_numpy(), bar_0['dy'], (grid_x, grid_y)).transpose()]],
                    label='y',
                    method='restyle')
            ]),
            direction='right', pad={'r': 10, 't': 10}, showactive=True, x=0.5, xanchor='left', y=1.1,
            yanchor='bottom', type='buttons', font=dict(size=13)
        ),
    ]
)

# Add annotation for button
fig.add_annotation(dict(font=dict(size=13), x=0.5, y=1.13, showarrow=False,
                   xref='paper', yref='paper', xanchor='right', yanchor='bottom', text='Displacement: '))

# update xaxis range and show figure
fig.update_xaxes(range=(-0.2,10.2), constrain='domain')
fig.show()

Select five measurement locations that you expect to be informative. Plug them into the predefined list `measure_locs`. Remember that we only have a budget of five locations, make sure to not exceed this threshold to secure a spot on the leaderboard (more to that later). The remaining code in this cell collects the displacements from all of our beams at the selected nodes and the defect location. These quantities are stored in the arrays `measurements` and `defect_locs`.

In [ ]:
# define measurement locations, get corresponding coord
# ----------------------------------------
measure_locs = [115, 180, 25, 372, 425] # <- fill in the indices of the 5 points you select
# ----------------------------------------

measure_coords = np.array([bar_0[bar_0['node'] == loc][['x','y']].to_numpy() for loc in measure_locs]).squeeze(1)

# double check the measurement locations
print(measure_coords)
                             
# read measurement from all samples in dataframe
measurements = np.empty((df['sample'].max()+1,0))

# loop through measurement locations and collect measuments from all samples
for loc in measure_locs:
    dx = df[df['node'] == loc]['dx'].to_numpy()
    dy = df[df['node'] == loc]['dy'].to_numpy()
    measurements = np.append(measurements, np.vstack((dx, dy)).transpose(),axis=1)
    
measurements_noisy = measurements + np.random.randn(*measurements.shape) * .5e-5

# get defect locations
defect_locs = df[df['node']==0]['location'].to_numpy()

Let's plot the defect location as a function of the displacement measurements to get a feel for the dataset:

In [ ]:
# plot a few hyperplanes of the dataset
fig, ax = plt.subplots(5,2, figsize=(10,20))
[ax.flat[i].scatter(measurements_noisy[:,i], defect_locs, s=5) for i in range(len(ax.flat))]
[ax.flat[i].ticklabel_format(style='sci', axis='x', scilimits=(0,0)) for i in range(len(ax.flat))]
[ax[0,i].set_title(title) for i,title in enumerate([r'$u_x$',r'$u_y$'])]
[ax[i,0].text(-0.4, 0.46, f'node {measure_locs[i]}', transform=ax[i,0].transAxes, fontsize=12) for i in range(ax.shape[0])]
[ax[i,0].set_ylabel(r'$x_{defect}$') for i in range(ax.shape[0])]
plt.show()

We can see that most measurements do not have a unique mapping to the defect location, suggesting we need more features to distinguish between the deformation states. Let's take a look at a 2D scatterplot of our data. Note that this is a projection of the data on this particular 2D subspace of the input space. The color bar indicates the defect location of a data point. This looks more promising; the defect location seems to be an injective function when considering multiple measurements.

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
plot1 = ax.scatter(measurements_noisy[::5,3], measurements_noisy[::5,7], c=defect_locs[::5], s=40)
[ax.ticklabel_format(style='sci', axis=axis, scilimits=(0,0)) for axis in ['x','y']]
fig.colorbar(plot1)
plt.show()

## Neural Network Training

We provide you with a function to train the network, but it only offers a bare minimum of features. You can train the network with this default training loop, but this is neither an efficient nor a robust way to do so. Consider the techniques we discussed in the notebooks to control the model complexity. You can extend the loop with features such as early stopping or L2-regularization (you can pass a regularizing parameter `alpha` to the `partial_fit` member function of the `NN` object. You can also tune the network architecture by adapting the number of hidden layers and the number of neurons per layer in the next cell. Try to justify your specific modeling choices.

In [ ]:
# function to train the NN
def NN_train(NN, X_train, y_train, X_val, y_val, max_epoch=10000, verbose=False, lr_init=1e-3, ):
    
    # set learning rate
    lr = lr_init
    NN.learning_rate_init = lr
    
    # loop over iterations
    for epoch in range(max_epoch):
        
        # train for one epoch, compute rmse on validation set
        NN.partial_fit(X_train, y_train)
        
        # print loss (optional)
        if verbose and epoch%200==0:
            print("\nIteration {}".format(epoch))
    
    if (epoch==max_epoch-1): print("Reachead max_epochs ( {} )".format(max_epoch))
    
    # return trained network and last rmse
    return NN

In [ ]:
# set random seed
np.random.seed(1)
indices = np.arange(measurements_noisy.shape[0])

# set up scalers and scale data
xscaler, yscaler = StandardScaler(), StandardScaler()
yit = yscaler.inverse_transform
xit = xscaler.inverse_transform
X, y = xscaler.fit_transform(measurements_noisy), yscaler.fit_transform(defect_locs[:,None]).reshape(-1)
X_train, X_test, y_train, y_test, ind_train, ind_test = train_test_split(X, y, indices, train_size=0.6)
X_val, X_test, y_val, y_test, ind_val, ind_test = train_test_split(X_test, y_test, ind_test, train_size=0.5)

# Set up NN
NN = MLPRegressor(solver='sgd', hidden_layer_sizes=(5, 5), activation='tanh')

# train NN
NN = NN_train(NN, X_train, y_train, X_val, y_val, max_epoch=10000, verbose=True)

# Visualizing result
You can use the following plotting routines to visualize your predictions. Keep in mind that all of the following graphs are based on projections of the input data on 1D or 2D subspaces that suppress at least part of the information contained in the dataset. Those projections, however, are necessary to enable visualizations of the predictions.

In [ ]:
# y_pred = NN.predict(X_test)
y_pred = NN.predict(X_test)
fig, ax = plt.subplots(figsize = (6,5))
loc = 3
ax.scatter(xit(X_test)[:,loc], yit(y_test[:,None]).reshape(-1), label='truth', s=30)
ax.scatter(xit(X_test)[:,loc], yit(y_pred[:,None]).reshape(-1), label='predition', s=30)
ax.legend()
ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
ax.set_xlabel(r'$u$')
ax.set_ylabel(r'$x_{defect}$')
plt.show()

In [ ]:
# plot prediciton for projection of inputs on 2D subspace
fig, ax = plt.subplots(1,3,figsize=(12,3.8), constrained_layout=True, sharey=True)
plot0 = ax[0].scatter(xit(X_test)[:,3], xit(X_test)[:,7], c=yit(y_test[:,None]).reshape(-1))
plot1 = ax[1].scatter(xit(X_test)[:,3], xit(X_test)[:,7], c=yit(y_pred[:,None]).reshape(-1))
plot2 = ax[2].scatter(xit(X_test)[:,3], xit(X_test)[:,7], c=np.abs(y_pred-y_test), cmap='Reds', vmin=0, vmax=0.2)
[plt.colorbar(plot, ax=ax[i]) for i, plot in enumerate([plot0, plot1, plot2])]
[ax[i].set_title(title) for i, title in enumerate([r'truth $y$', r'prediction $\hat y$', r'$|y - \hat y|$'])]
[axs.ticklabel_format(style='sci', axis=axis, scilimits=(0,0)) for axis in ['x','y'] for axs in ax]
plt.show()

You can pick a few datapoints (or samples/bridges) from the test set to inspect how well our predictions compare with the ground truth. Do this by varying the index value in the code below.

In [ ]:
# extract data corresponding to one single bridge, an interpolate the displacements on a grid for plotting
# ----------------------------
index = 56  # <- Change this value to plot different samples!
# ----------------------------

total_idx = ind_test[index]

# get corresponding bar, displacement fields and measurement location coordinates
bar = df[df['sample'] == total_idx]
measure_coords = np.array([bar[bar['node'] == loc][['x','y']].to_numpy() for loc in measure_locs]).squeeze(1)
grid_x, grid_y = np.mgrid[0.02:9.98:250j, 0.02:1.98:50j]
grid_z = griddata(bar[['x','y']].to_numpy(), np.sqrt(bar['dx']**2 + bar['dy']**2), (grid_x, grid_y))

# # plot displacement field and nodes
fig = go.Figure()
fig.add_trace(go.Heatmap(z=grid_z.transpose(), x=grid_x[:,0], y=grid_y[0],
                         hoverinfo='skip', name='heatmap'))

# add buttons for additinal fields
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=['z', [griddata(bar[['x','y']].to_numpy(), np.sqrt(bar['dx']**2 + bar['dy']**2), (grid_x, grid_y)).transpose()]],
                    label='magnitude', method='restyle'),
                dict(
                    args=['z', [griddata(bar[['x','y']].to_numpy(), bar['dx'], (grid_x, grid_y)).transpose()]],
                    label='x',
                    method='restyle'),
                dict(
                    args=['z', [griddata(bar[['x','y']].to_numpy(), bar['dy'], (grid_x, grid_y)).transpose()]],
                    label='y',
                    method='restyle')
            ]),
            direction='right', pad={'r': 10, 't': 10}, showactive=True, x=0.5, xanchor='left', y=1.2,
            yanchor='bottom', type='buttons', font=dict(size=13)
        ),
    ]
)

# Add annotation for button
fig.add_annotation(dict(font=dict(size=13), x=0.5, y=1.23, showarrow=False,
                   xref='paper', yref='paper', xanchor='right', yanchor='bottom', text='Displacement: '))

# plot measurement locations
fig.add_trace(go.Scatter(x = measure_coords[:,0], y = measure_coords[:,1], mode='markers',
                         marker=dict(size=10, color='DarkSlateGrey', line=dict(width=2, color='white')),
                         hovertemplate='<b>Node</b>: %{text}', text=measure_locs, name=''))

# get prediction and true value for defect location
defect_loc_pred = yit(NN.predict(X_test[[index],:])[:,None])[0,0]
defect_loc_true = yit(y_test[[index]][None,:])[0,0]

# plot vertical lines at the two locations
fig.add_vline(x=defect_loc_pred, name='pred', line=dict(color='LightSlateGrey'))
fig.add_vline(x=defect_loc_true, name='truth', line=dict(color='LightSlateGrey'), line_dash='dot')

# put labels on the lines
fig.add_annotation(dict(font=dict(size=13), x=defect_loc_pred, y=1.15, showarrow=False,
                   xref='x', yref='paper', text='prediction: {:.2f}'.format(defect_loc_pred)))
fig.add_annotation(dict(font=dict(size=13), x=defect_loc_true, y=-.22, showarrow=False,
                   xref='x', yref='paper', text='truth: {:.2f}'.format(defect_loc_true)))
                        
# update axis and show figure
fig.update_xaxes(range=(-0.2,10.2), constrain='domain')
fig.show()

Finally, you need to compute the RMSE for all samples in the test set to quantify our accuracy. Show us your implementation - If you get a new high score, we'll put you on top of the leaderboard!

In [ ]:
y_pred_test = NN.predict(X_test)
rmse_test = np.sqrt(np.sum((yit(y_pred_test[:,None]) - yit(y_test[:,None])).reshape(-1)**2) / y_test.shape[0])
print("RMSE on test set for best performing model: {:.4e}".format(rmse_test))